In [1]:
import os
import pathlib
import dspy
from dspy import Example
from dspy.evaluate.evaluate import Evaluate
from dspy.evaluate.metrics import answer_exact_match
from dspy import Signature, InputField, OutputField
from pydantic import BaseModel, Field
from datetime import datetime
from dspy.teleprompt import MIPRO, BootstrapFewShotWithRandomSearch

In [2]:
class Problem(BaseModel):
    problem_dir: pathlib.Path = Field(
        ..., description="The path to the problem directory"
    )
    problem_name: str = Field(..., description="The name of the problem")
    problem_description: str = Field(..., description="The description of the problem")
    sample_input: str = Field(..., description="The sample input of the problem")
    sample_output: str = Field(..., description="The sample output of the problem")
    problem_input: pathlib.Path = Field(..., description="The path to the input file")
    problem_output: pathlib.Path = Field(..., description="The path to the output file")
    solution: str = Field(..., description="The solution to the problem")

    @property
    def as_xml(self) -> str:
        return f"""
<problem>
<problem_statement>
{self.problem_description}
</problem_statement>
<sample_test_cases>
<sample_input>
{self.sample_input}
</sample_input>
<sample_output>
{self.sample_output}
</sample_output>
</sample_test_cases>
</problem>
"""


def load_problem(problem_name: str, problem_dir: pathlib.Path) -> Problem:
    problem_input = problem_dir / f"{problem_name}.in"
    problem_output = problem_dir / f"{problem_name}.out"
    sample_input = problem_dir / f"{problem_name}_sample_input.txt"
    sample_output = problem_dir / f"{problem_name}_sample_output.txt"
    problem_description = problem_dir / f"{problem_name}.md"
    solution = problem_dir / f"{problem_name}_sol.md"
    return Problem(
        problem_dir=problem_dir,
        problem_name=problem_name,
        problem_description=problem_description.read_text(),
        sample_input=sample_input.read_text(),
        sample_output=sample_output.read_text(),
        problem_input=problem_input,
        problem_output=problem_output,
        solution=solution.read_text(),
    )

In [3]:
problems = list(pathlib.Path("dataset/").rglob("*.in"))
eval_problems = list(filter(lambda x: "2022/final" in str(x), problems))
test_problems = list(filter(lambda x: "2023/practice" in str(x), problems))
train_problems = list(filter(lambda x: x not in eval_problems + test_problems, problems))


def load_problem_from_path(problem_path: pathlib.Path):
    problem_name = problem_path.stem
    problem_dir = problem_path.parent
    try:
        problem = load_problem(problem_name, problem_dir)
    except Exception as e:
        print(f"Error loading problem {problem_name}: {e}")
        return None
    return problem



# in 2022 there are some problems that are missing the sample input and output
train_problems = list(l for l in map(load_problem_from_path, train_problems) if l is not None)
eval_problems = list(l for l in map(load_problem_from_path, eval_problems) if l is not None)
test_problems = list(l for l in map(load_problem_from_path, test_problems) if l is not None)


Error loading problem zero_crossings_ch1: [Errno 2] No such file or directory: 'dataset/2022/round3/zero_crossings_ch1.md'


In [4]:
(len(train_problems), len(eval_problems), len(test_problems))

(44, 6, 5)

In [5]:
trainset = [Example(
    problem_statement=problem.problem_description,
    sample_input=problem.sample_input,
    sample_output=problem.sample_output,
    solution=problem.solution).with_inputs("problem_statement", "sample_input", "sample_output") for problem in train_problems]

devset = [Example(
    problem_statement=problem.problem_description,
    sample_input=problem.sample_input,
    sample_output=problem.sample_output,
    solution=problem.solution).with_inputs("problem_statement", "sample_input", "sample_output") for problem in eval_problems]

In [6]:
import weave
weave.init("dspy_hackercup")

MODEL = "mistral-large-latest"

from dsp import LM
from typing import Any
import json
from mistralai import Mistral  # requires mistralai>1.0

class MistralLM(LM):

    def __init__(self, model, api_key, **kwargs):
        self.model = model
        self.api_key = api_key
        self.client = Mistral(api_key=api_key)
        self.kwargs = {
            "temperature": 0.17,
            "max_tokens": 150,
            **kwargs,
        }
        self.history: list[dict[str, Any]] = []
        self.provider = "mistralai"

    def basic_request(self, prompt: str, **kwargs):

        # Mistral disallows "n" arguments
        n = kwargs.pop("n", None)
        if n is not None and n > 1 and kwargs["temperature"] == 0.0:
            kwargs["temperature"] = 0.7

        chat_response = self.client.chat.complete(
            model=self.model,
            messages=[
                dict(role="user", content=prompt)
            ],
            **kwargs
        )
        response_dict = json.loads(chat_response.model_dump_json())

        self.history.append({
            "prompt": prompt,
            "response": response_dict, #{"choices": chat_response.choices[0].message.content},
            "kwargs": kwargs,
        })

        return chat_response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.request(prompt, **kwargs)
        completions = [response.choices[0].message.content] 
        return completions

    def _get_choice_text(self, choice: dict[str, Any]) -> str:

        return choice["message"]["content"]

weave version 0.51.7 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/dspy_hackercup/weave


In [7]:
mistral = MistralLM(model=MODEL, api_key=os.environ["MISTRAL_API_KEY"])

Let's test the endpoint

In [8]:
mistral("hello")

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffeb-4003-7b91-85d4-adf0c39dd398


['Hello! How are you today? How can I assist you? 😊']

In [20]:
dspy.settings.configure(lm=mistral)
dspy.configure(experimental=True)


@weave.op
def validate_solution(example, prediction, trace=None, frac=0.8):
    
    result = answer_exact_match(
        example=Example(answer=example.solution),
        pred=Example(answer=prediction.solution),
        trace=trace, 
        frac=frac)
    return result

from dsp.utils import F1

def f1(example, prediction, trace=None,):
    return F1(example.solution, [prediction.solution])

# Setup evaluation function
evaluate = Evaluate(
    devset=devset,
    num_threads=6, # Note: Set this to 1 for debugging purposes 
    display_progress=True,
    display_table=5,
    metric=f1
)

Let's understand dspy.answer_match and the underlying F1 metric

In [21]:
from dspy import dsp


dsp.answer_match("You are helpful", ["You are not helpful in you work"], frac=0.8)

False

In [22]:
F1("You are helpful", ["You are not helpful in you work"])

0.6

In [23]:
class GenerateSolution(Signature):
    """You are an expert problem solver. Your task is to solve the problem at hand.

    When solving a competitive programming problem, start by thoroughly reading and understanding the problem statement, including all constraints and input/output formats.
    Identify the key elements, analyze sample inputs/outputs, and consider edge cases.
    Look for patterns or mathematical relationships, then develop a logical approach, breaking the problem into subproblems if needed. 
    Explain the core insight in simple terms, followed by a step-by-step explanation of the solution.
    Use clear language and visual aids if helpful. Discuss optimization techniques, alternative approaches, and special cases. 
    Address time and space complexity, and provide pseudocode if appropriate. 
    Finally, proofread the solution for clarity and completeness, ensuring it's accessible to readers with varying levels of programming experience.
    Note: You are not expected to write the code for the solution, just provide a solution explanation so that an experienced developer can write the code for the solution.
    """

    problem_statement: str = InputField(format=str)
    sample_input: str = InputField(format=str, desc="The sample input provided with the problem statement.")
    sample_output: str = InputField(format=str, desc="The sample output provided with the problem statement.")
    solution: str = OutputField(
        format=str, desc="a solution explanation for how we should go about solving this problem."
    )

In [13]:
dspy.ChainOfThought(GenerateSolution)(
    problem_statement=trainset[0].problem_statement, 
    sample_input=trainset[0].sample_input, 
    sample_output=trainset[0].sample_output
    ).solution


🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffeb-8ffd-7851-8c22-73766c3832e0


"### Problem Statement:\nYou have \\(N\\) tries. The \\(i\\)th trie has \\(M_i\\) nodes and \\(M_i - 1\\) edges, with node \\(1\\) being the root, and node \\(j\\)'s parent being \\(P_{i,j}\\), for \\(j = 2.."

In [24]:
class SimpleGenerateSolution(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_code = dspy.ChainOfThought(GenerateSolution)

    def forward(self, problem_statement, sample_input, sample_output):
        solution = self.generate_code(
                problem_statement=problem_statement,
                sample_input=sample_input,
                sample_output=sample_output
            ).solution

        return dspy.Prediction(solution=solution)


In [25]:
simple_program = SimpleGenerateSolution()

In [16]:
evaluate(program=simple_program, devset=devset)


Average Metric: 0.048109965635738834 / 1  (4.8):  17%|█▋        | 1/6 [00:36<03:00, 36.15s/it]

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffec-9f54-7af2-96b4-c4831b230b62


Average Metric: 0.15945017182130586 / 2  (8.0):  33%|███▎      | 2/6 [00:40<01:09, 17.41s/it] 

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffec-9f4b-7730-afe5-0d096b512415


Average Metric: 0.4336640082992933 / 3  (14.5):  50%|█████     | 3/6 [00:46<00:36, 12.13s/it]

...

Average Metric: 0.5900078507162558 / 6  (9.8): 100%|██████████| 6/6 [01:14<00:00, 12.46s/it] 

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffec-a01b-7c02-af04-9ed6236f10d2
Average Metric: 0.5900078507162558 / 6  (9.8%)


,problem_statement,sample_input,sample_output,example_solution,pred_solution,f1
0,"Ishiko is opening a bracelet shop. Each bracelet she plans to sell is expressed as a ring of \(N\) colored beads, with exactly \(K\) green...",4 5 3 6 3 6 2 243447 42273,Case #1: 1 Case #2: 2 Case #3: 4 Case #4: 4,"First we need to determine how many bracelets Ishiko is going to exhibit. When \(N = K\) (that is, when all of the beads are...","### Problem Statement: Ishiko is opening a bracelet shop. Each bracelet she plans to sell is expressed as a ring of \(N\) colored beads, with...",0.11134020618556702
1,"Boss Rob planted \(N\) happy little hazel trees in his yard (represented on a Cartesian plane). The \(i\)th tree is at integer coordinates \((X_i, Y_i)\)....",3 4 0 0 2 1 5 0 0 6 5 10 10 12 10 8 10 5 10 8 8 4 1 1 3...,Case #1: 20 Case #2: 28 Case #3: 42,"First, we'll note a greedy philosophy that if we can simply use a \(2 \times 2\) square around all trees, that’d be the best solution...",### Problem Statement: Boss Rob planted \(N\) happy little hazel trees in his yard (represented on a Cartesian plane). The \(i\)th tree is at integer...,0.048109965635738834
2,Alphonse is assembling an *alphabet tree* and arranging some adventures along the way. An alphabet tree is an unrooted tree with \(N\) nodes (numbered from...,3 9 1 2 M 1 3 E 1 4 T 4 9 A 2 5 T 2 6 E 3 7 A 3 8...,Case #1: 6 9 2 9 10 Case #2: 2 1 6 4 Case #3: 1 1 2,"The first thing to do is root the tree, for which we can arbitrarily choose node \(1\). Each journey can be done in two phases:...",### Problem Statement: Alphonse is assembling an *alphabet tree* and arranging some adventures along the way. An alphabet tree is an unrooted tree with \(N\)...,0.05263157894736842
3,"Courtney has an avant-garde kitchen counter made out of \(N\) rectangular sheets of metal of equal height, standing perpendicular to the ground. When viewed directly...",2 3 1 4 0 0 2 2 4 0 5 1 4 0 0 1 2 0 3 4 3 3 1,Case #1: 0.1169663730642699 Case #2: 0.1353445414060363,"Let's call a position where the circle doesn't fall *stable*. First, let's see how we can check whether a given point is stable. Consider all...",1. **Understand the Geometry**: - The problem involves a polygon defined by \(N\) edges. - The cup is modeled as a circle with radius \(R\)....,0.27421383647798747
4,"You just landed a job as a machine learning engineer! As a ramp-up exercise, Boss Rob tasked you with modeling the watering wells in his...",2 500 18.243577 16.343618 24.560940 7.478552 13.664297 0.348593 19.766713 16.871980 14.052491 10.567715 21.426414 5.786941 20.495098 -0.246197 20.706538 14.324784 13.240629 9.591812 18.131521 1.645394 13.085966 5.206907 12.705525...,Case #1: 6 10 30 2 19 Case #2: 40 50 38 45 13,"There are many ways you might approach this problem, but a simple one is to find the pair of circles that contain all the given...","### Solution Explanation To solve the problem of predicting the exact values of \(A_x\), \(A_y\), \(B_x\), \(B_y\), and \(R\) based on the planted tree coordinates...",0.06644518272425248


🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffec-9c77-70d2-9b21-cfe2f867bb1b


9.83

In [26]:
def optimize_with_bootstrap_fewshot(program, task_model, teacher_model, metric, trainset):
    rs_optimizer = BootstrapFewShotWithRandomSearch(
        metric=metric,
        num_threads=8,
        num_candidate_programs=2,
        max_labeled_demos=0,
        max_bootstrapped_demos=2,
        max_errors=10000,
        teacher_settings=dict(lm=teacher_model)
    )
    
    optimized_program = rs_optimizer.compile(
        program,
        trainset=trainset,
    )

    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")

    optimized_program.save(f"fewshot_optimized_{date_time}")


    return optimized_program

In [27]:
optimized_program = optimize_with_bootstrap_fewshot(simple_program, mistral, mistral, f1, trainset[0:5])

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 2 candidate sets.


Average Metric: 0.07500000000000001 / 1  (7.5):  20%|██        | 1/5 [00:37<02:29, 37.27s/it]

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffff-3e04-73b2-acfe-8b880075dd5e


Average Metric: 0.137015503875969 / 2  (6.9):  40%|████      | 2/5 [00:44<00:57, 19.32s/it]  

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffff-3e0a-7ff3-9de2-e6a8fe141e02


...

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0192000e-1786-7531-b215-f080751bfbef


Average Metric: 0.3514108496149412 / 4  (8.8):  80%|████████  | 4/5 [01:09<00:12, 12.55s/it] 

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0192000e-1756-7601-bdaf-568d5710bda5


Average Metric: 0.42352623423032576 / 5  (8.5): 100%|██████████| 5/5 [01:10<00:00, 14.02s/it]

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0192000e-175c-7302-b28c-334da9cf2345
Average Metric: 0.42352623423032576 / 5  (8.5%)
Score: 8.47 for set: [1]
Scores so far: [7.56, 13.86, 5.71, 19.1, 8.47]
Best score: 19.1
Average of max per entry across top 1 scores: 0.1909534773918633
Average of max per entry across top 2 scores: 0.19715426110866333
Average of max per entry across top 3 scores: 0.20753497176856178
Average of max per entry across top 5 scores: 0.20753497176856178
Average of max per entry across top 8 scores: 0.20753497176856178
Average of max per entry across top 9999 scores: 0.20753497176856178
5 candidate programs found.
🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/0191ffff-3e01-7e80-a448-ebc211783b0e


In [28]:
optimized_program

generate_code = ChainOfThought(GenerateSolution(problem_statement, sample_input, sample_output -> solution
    instructions="You are an expert problem solver. Your task is to solve the problem at hand.\n\n    When solving a competitive programming problem, start by thoroughly reading and understanding the problem statement, including all constraints and input/output formats.\n    Identify the key elements, analyze sample inputs/outputs, and consider edge cases.\n    Look for patterns or mathematical relationships, then develop a logical approach, breaking the problem into subproblems if needed. \n    Explain the core insight in simple terms, followed by a step-by-step explanation of the solution.\n    Use clear language and visual aids if helpful. Discuss optimization techniques, alternative approaches, and special cases. \n    Address time and space complexity, and provide pseudocode if appropriate. \n    Finally, proofread the solution for clarity and completeness, ensuring it's acce

In [29]:
evaluate(program=optimized_program, devset=devset)

Average Metric: 0.11134020618556702 / 1  (11.1):  17%|█▋        | 1/6 [00:42<03:33, 42.78s/it]

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/01920010-0e28-7053-8db3-16f82422ac7f


Average Metric: 0.1526625202351538 / 2  (7.6):  33%|███▎      | 2/6 [00:56<01:42, 25.50s/it]  

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/01920010-0e70-77d1-aae1-9a44f8d1abfd


Average Metric: 0.21462038888447227 / 3  (7.2):  50%|█████     | 3/6 [01:02<00:50, 16.82s/it]

...

Average Metric: 0.37297171977800775 / 6  (6.2): 100%|██████████| 6/6 [01:20<00:00, 13.42s/it]

🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/01920010-0e51-7b60-a445-0d5b6e870d08
Average Metric: 0.37297171977800775 / 6  (6.2%)


,problem_statement,sample_input,sample_output,example_solution,pred_solution,f1
0,"Ishiko is opening a bracelet shop. Each bracelet she plans to sell is expressed as a ring of \(N\) colored beads, with exactly \(K\) green...",4 5 3 6 3 6 2 243447 42273,Case #1: 1 Case #2: 2 Case #3: 4 Case #4: 4,"First we need to determine how many bracelets Ishiko is going to exhibit. When \(N = K\) (that is, when all of the beads are...","### Problem Statement: Ishiko is opening a bracelet shop. Each bracelet she plans to sell is expressed as a ring of \(N\) colored beads, with...",0.11134020618556702
1,"Boss Rob planted \(N\) happy little hazel trees in his yard (represented on a Cartesian plane). The \(i\)th tree is at integer coordinates \((X_i, Y_i)\)....",3 4 0 0 2 1 5 0 0 6 5 10 10 12 10 8 10 5 10 8 8 4 1 1 3...,Case #1: 20 Case #2: 28 Case #3: 42,"First, we'll note a greedy philosophy that if we can simply use a \(2 \times 2\) square around all trees, that’d be the best solution...","### Problem Statement: Boss Rob planted \(N\) happy little hazel trees in his yard, represented on a Cartesian plane. The \(i\)th tree is at integer...",0.048027444253859346
2,Alphonse is assembling an *alphabet tree* and arranging some adventures along the way. An alphabet tree is an unrooted tree with \(N\) nodes (numbered from...,3 9 1 2 M 1 3 E 1 4 T 4 9 A 2 5 T 2 6 E 3 7 A 3 8...,Case #1: 6 9 2 9 10 Case #2: 2 1 6 4 Case #3: 1 1 2,"The first thing to do is root the tree, for which we can arbitrarily choose node \(1\). Each journey can be done in two phases:...",### Problem Statement: Alphonse is assembling an *alphabet tree* and arranging some adventures along the way. An alphabet tree is an unrooted tree with \(N\)...,0.05263157894736842
3,"Courtney has an avant-garde kitchen counter made out of \(N\) rectangular sheets of metal of equal height, standing perpendicular to the ground. When viewed directly...",2 3 1 4 0 0 2 2 4 0 5 1 4 0 0 1 2 0 3 4 3 3 1,Case #1: 0.1169663730642699 Case #2: 0.1353445414060363,"Let's call a position where the circle doesn't fall *stable*. First, let's see how we can check whether a given point is stable. Consider all...","### Problem Statement: Courtney has an avant-garde kitchen counter made out of \(N\) rectangular sheets of metal of equal height, standing perpendicular to the ground....",0.04132231404958678
4,"You just landed a job as a machine learning engineer! As a ramp-up exercise, Boss Rob tasked you with modeling the watering wells in his...",2 500 18.243577 16.343618 24.560940 7.478552 13.664297 0.348593 19.766713 16.871980 14.052491 10.567715 21.426414 5.786941 20.495098 -0.246197 20.706538 14.324784 13.240629 9.591812 18.131521 1.645394 13.085966 5.206907 12.705525...,Case #1: 6 10 30 2 19 Case #2: 40 50 38 45 13,"There are many ways you might approach this problem, but a simple one is to find the pair of circles that contain all the given...","### Problem Statement: Boss Rob has a fence made of \(N\) wooden stakes, numbered \(1\) to \(N\). Initially, the \(i\)th stake is of color \(i\)....",0.0576923076923077


🍩 https://wandb.ai/capecape/dspy_hackercup/r/call/01920010-0e25-7bd0-8537-3fd3f96dcc27


6.22

In [33]:
def optimize_with_mipro(program, prompt_model, task_model, metric, trainset):
    teleprompter = MIPRO(
        prompt_model=prompt_model,
        task_model=task_model,
        metric=metric,
        num_candidates=2,
        init_temperature=0.5,
        verbose=False,
    )

    optimized_program = teleprompter.compile(
        program.deepcopy(),
        num_trials=10,
        trainset=trainset,
        eval_kwargs=dict(num_threads=8),
        max_bootstrapped_demos=0, # 0-shot optimization
        max_labeled_demos=0,
        seed=9
    )
    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")

    optimized_program.save(f"mipro_optimized_{date_time}")

    return optimized_program


mipro_optimized_program = optimize_with_mipro(simple_program, mistral, mistral, f1, trainset[0:5])


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 5 examples in dev set * 10 trials * # of LM calls in your program = (50 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 2 * 1 lm calls in program = 12 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:

- Redu

  0%|          | 0/5 [00:59<?, ?it/s]


KeyboardInterrupt: 

In [29]:
evaluate(program=mipro_optimized_program, devset=devset)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'You are a highly skilled algorithmic problem solver tasked with providing a comprehensive and detailed explanation for a competitive programming challenge. Begin by carefully analyzing the problem statement, paying close attention to constraints, input and output formats. Identify critical elements and analyze the provided sample inputs and outputs, while also considering edge cases that may arise. Look for underlying patterns or mathematical relationships that can guide your approach. Break down the problem into smaller, manageable subproblems if necessary, and articulate the core insights in straightforward terms. Provide a step-by-step explanation of your reasoning and proposed solution, using clear language and, if beneficial, visual aids to enhance understanding. Discuss potential optimization techniques, alternative metho

,problem_statement,sample_input,sample_output,example_solution,pred_solution,validate_solution
0,"Ishiko is opening a bracelet shop. Each bracelet she plans to sell is expressed as a ring of \(N\) colored beads, with exactly \(K\) green...",4 5 3 6 3 6 2 243447 42273,Case #1: 1 Case #2: 2 Case #3: 4 Case #4: 4,"First we need to determine how many bracelets Ishiko is going to exhibit. When \(N = K\) (that is, when all of the beads are...",The solution can be structured as follows: 1. **Calculate the Number of Distinct Bracelets**: - Implement the formula for counting distinct bracelets based on \(N\)...,False
1,"Boss Rob planted \(N\) happy little hazel trees in his yard (represented on a Cartesian plane). The \(i\)th tree is at integer coordinates \((X_i, Y_i)\)....",3 4 0 0 2 1 5 0 0 6 5 10 10 12 10 8 10 5 10 8 8 4 1 1 3...,Case #1: 20 Case #2: 28 Case #3: 42,"First, we'll note a greedy philosophy that if we can simply use a \(2 \times 2\) square around all trees, that’d be the best solution...","1. **Input Reading**: Read the number of test cases \(T\). For each test case, read the number of trees \(N\) and their coordinates. 2. **Sorting**:...",False
2,Alphonse is assembling an *alphabet tree* and arranging some adventures along the way. An alphabet tree is an unrooted tree with \(N\) nodes (numbered from...,3 9 1 2 M 1 3 E 1 4 T 4 9 A 2 5 T 2 6 E 3 7 A 3 8...,Case #1: 6 9 2 9 10 Case #2: 2 1 6 4 Case #3: 1 1 2,"The first thing to do is root the tree, for which we can arbitrarily choose node \(1\). Each journey can be done in two phases:...","To implement the solution, we can follow these steps: 1. **Data Structures**: - Use an adjacency list to represent the tree, where each node points...",False
3,"Courtney has an avant-garde kitchen counter made out of \(N\) rectangular sheets of metal of equal height, standing perpendicular to the ground. When viewed directly...",2 3 1 4 0 0 2 2 4 0 5 1 4 0 0 1 2 0 3 4 3 3 1,Case #1: 0.1169663730642699 Case #2: 0.1353445414060363,"Let's call a position where the circle doesn't fall *stable*. First, let's see how we can check whether a given point is stable. Consider all...","To solve the problem, we will follow these steps: 1. **Input Parsing**: Read the number of test cases \(T\). For each test case, read \(N\),...",False
4,"You just landed a job as a machine learning engineer! As a ramp-up exercise, Boss Rob tasked you with modeling the watering wells in his...",2 500 18.243577 16.343618 24.560940 7.478552 13.664297 0.348593 19.766713 16.871980 14.052491 10.567715 21.426414 5.786941 20.495098 -0.246197 20.706538 14.324784 13.240629 9.591812 18.131521 1.645394 13.085966 5.206907 12.705525...,Case #1: 6 10 30 2 19 Case #2: 40 50 38 45 13,"There are many ways you might approach this problem, but a simple one is to find the pair of circles that contain all the given...","1. Calculate the centroid of the tree coordinates to estimate \((A_x, A_y)\). 2. For each tree, compute the distance from the centroid to the tree's...",False


0.0

## Using the optimized program

We have developed a program that can solve the problem by generating a solution (not the code, but the logic).

Now we can use this solution to create the code for each sample problem.

In [34]:
# TODO